# <center> <b> <span style="color:RED;"> TOPOLOGICAL DATA ANALYSIS </span> </b></center>


## <center> <b> <span style="color:blue;"> GROUP 11  </span> </b></center>

## <center> <b> <span style="color:orange;">Project 1 </span> </b></center>

## <b> <span style="color:green;">GROUP MEMBERS: </span> </b>
 - Emmanuel Ansah
 - Linda INGABIRE
 - Josue TWAHIRWA
 - Miarisoa Elalie RASAMIMANANA

# Incremental Algorithm

In [1]:
import numpy as np
from copy import deepcopy
def incremental_betti(simplices):
    """Computes Betti numbers using the incremental algorithm."""
    sorted_simplices = sort_simplices(simplices)
    betti_numbers = {}
    max_dim = max(len(s) for s in sorted_simplices) - 1
    for dim in range(max_dim + 1):
        betti_numbers[dim] = 0

    boundary_matrix = create_boundary_matrix(sorted_simplices)
    reduced_matrix = reduce_matrix(boundary_matrix, verbose=False) # Turn on verbose for debugging

    for j, simplex in enumerate(sorted_simplices):
        dim = len(simplex) - 1
        if np.all(reduced_matrix[:, j] == 0):
            betti_numbers[dim] += 1
        elif dim > 0:
            betti_numbers[dim - 1] -= 1
    betti_numbers_list = [betti_numbers[i] for i in sorted(betti_numbers)]
    return betti_numbers_list

# Gauss Reduction Algorithm

In [2]:
def sort_simplices(simplices):
    """Sorts simplices by dimension and then lexicographically."""
    return sorted(simplices, key=lambda s: (len(s), tuple(s)))

def create_boundary_matrix(simplices):
    """Creates the boundary matrix for the given simplices."""
    n = len(simplices)
    matrix = np.zeros((n, n), dtype=int)
    for j, simplex_j in enumerate(simplices):
      for i, simplex_i in enumerate(simplices):
          if len(simplex_j) == len(simplex_i) + 1 and set(simplex_i).issubset(set(simplex_j)):
              matrix[i,j] = 1
    return matrix


def reduce_matrix(matrix, verbose=False):
    """Performs Gaussian reduction on the boundary matrix."""
    reduced_matrix = deepcopy(matrix)
    num_cols = reduced_matrix.shape[1]
    pivots = {}
    if verbose:
        print("Initial Matrix:")
        print(reduced_matrix)
    for j in range(num_cols):
      column = reduced_matrix[:,j]
      pivot_rows = np.where(column == 1)[0]
      if len(pivot_rows) > 0:
        pivots[j] = max(pivot_rows)
    if verbose:
      print("Initial pivots:", pivots)
    j=0
    while j < num_cols:
        current_column = reduced_matrix[:, j]
        pivot_rows = np.where(current_column == 1)[0]
        if len(pivot_rows) > 0:
            pivot_row = max(pivot_rows) #get the pivot index, for column j
            for k in range(j):
                other_column = reduced_matrix[:, k]
                other_pivot_rows = np.where(other_column == 1)[0]
                if len(other_pivot_rows) > 0:
                  other_pivot_row = max(other_pivot_rows)
                  if other_pivot_row == pivot_row:
                    reduced_matrix[:,j] = (reduced_matrix[:,j] + reduced_matrix[:,k]) % 2
                    pivot_rows = np.where(reduced_matrix[:,j] == 1)[0]
                    new_pivots = {}
                    if len(pivot_rows) > 0:
                      new_pivots[j] = max(pivot_rows)
                    pivots.update(new_pivots)
                    if verbose:
                        print(f"Added column {k} to column {j}, updated pivots:")
                        print(reduced_matrix)
                        print(pivots)
                    break # go to next column
            else: # if we didn't find any other columns with same pivot, we go to the next column
              j+=1
        else: #if no pivot in current column, go to the next one
            j+=1
    if verbose:
      print("Final pivots:", pivots)
    return reduced_matrix

## Testing for circle

In [3]:
circle = [(0,), (1,), (2,), (0, 1), (1, 2), (0, 2)]
print("Betti numbers of the Circle:", incremental_betti(circle))

s=sort_simplices(circle)

b=create_boundary_matrix(s)

reduce_matrix(b)


Betti numbers of the Circle: [1, 1]


array([[0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

## Testing for sphere

In [4]:
sphere = [(0,), (1,), (2,), (3,), (0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]
print("Betti numbers of the Sphere:", incremental_betti(sphere))

s=sort_simplices(sphere)
b=create_boundary_matrix(s)
reduce_matrix(b)

Betti numbers of the Sphere: [1, 0, 1]


array([[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

## Testing for Torus

In [5]:

torus = [
    # 0-simplices (vertices)
    (0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,),

    # 1-simplices (edges)
    (0, 1), (0, 2), (0, 3), (0, 4), (0, 6), (0, 7),
    (1, 2), (1, 3), (1, 5), (1, 6), (1, 8),
    (2, 5), (2, 7), (2, 8), (2, 4),
    (3, 4), (3, 5), (3, 7), (3, 8),
    (4, 5), (4, 6), (4, 8),
    (5, 6), (5, 7),
    (6, 7), (6, 8),
    (7, 8),

    # 2-simplices (faces)
    (0, 1, 3), (1, 3, 5), (1, 2, 5), (2, 5, 7), (0, 2, 7), (0, 3, 7),
    (3, 4, 5), (4, 5, 6), (5, 6, 7), (6, 7, 8), (3, 7, 8), (3, 4, 8),
    (0, 4, 6), (0, 1, 6), (1, 6, 8), (1, 2, 8), (2, 8, 4), (0, 2, 4)
]

print("Betti numbers of the Torus:", incremental_betti(torus))

s=sort_simplices(torus)
b=create_boundary_matrix(s)
reduce_matrix(b)

Betti numbers of the Torus: [1, 2, 1]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])